In [1]:
!pip install gymnasium
!pip install pygame

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/965.4 kB ? eta -:--:--
   ---------------------------------------- 965.4/965.4 kB 9.0 MB/s eta 0:00:00



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: C:\Users\dario\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
  Using cached pygame-2.6.1-cp312-cp312-win_amd64.whl.metadata (13 kB)
Using cached pygame-2.6.1-cp312-cp312-win_amd64.whl (10.6 MB)



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: C:\Users\dario\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


# Carga y exploración de un entorno de Gymnasium

In [2]:
import gymnasium as gym
import numpy as np
import time
from IPython.display import clear_output, display, HTML
import matplotlib.pyplot as plt
from tqdm import tqdm
import pandas as pd
import random
from itertools import product
from concurrent.futures import ProcessPoolExecutor, as_completed
import imageio
from IPython.display import HTML

SEED = 42

# -------------------- VISUALIZACIÓN DE RESULTADOS --------------------

def graficar_recompensas(agente):
    """Grafica la recompensa media acumulada por episodio."""
    plt.figure(figsize=(6, 3))
    plt.plot(agente.stats)
    plt.title('Recompensa media acumulada')
    plt.xlabel('Episodio')
    plt.ylabel('Recompensa media')
    plt.grid(True)
    plt.show()

def graficar_longitud_episodios(agente):
    """Grafica la longitud de cada episodio."""
    plt.figure(figsize=(6, 3))
    plt.plot(agente.episode_lengths)
    plt.title("Longitud de episodios")
    plt.xlabel("Episodio")
    plt.ylabel("Pasos")
    plt.grid(True)
    plt.show()

def mostrar_resultados_agente_continuo(agente):
    """Muestra gráficos de rendimiento en entornos continuos."""
    graficar_recompensas(agente)
    graficar_longitud_episodios(agente)

# -------------------- EJECUCIÓN DE UN EPISODIO --------------------

def ejecutar_episodio_y_mostrar(agente, render=False):
    """Ejecuta un episodio con la política aprendida y muestra la evolución de la posición."""
    env = agente.env
    state, _ = env.reset()
    done = False
    total_reward = 0
    posiciones = []

    while not done:
        if render:
            env.render()
        posiciones.append(state[0])  # Guardamos la posición del coche
        action = agente._seleccionar_accion_rbf(state)
        state, reward, terminated, truncated, _ = env.step(action)
        done = terminated or truncated
        total_reward += reward

    env.close()
    
    # Mostrar gráfico de posiciones
    plt.figure(figsize=(6, 3))
    plt.plot(posiciones)
    plt.title("Evolución de la posición del coche")
    plt.xlabel("Paso del episodio")
    plt.ylabel("Posición")
    plt.grid(True)
    plt.show()
    
    print(f"Recompensa total obtenida: {total_reward:.2f}")

# -------------------- VISUALIZACIÓN DE LA POLÍTICA EN EL ESPACIO CONTINUO --------------------

def visualizar_politica_aprendida(agente, resolution=50):
    """Visualiza la política aprendida en el espacio de estados continuo."""
    x = np.linspace(agente.low[0], agente.high[0], resolution)  # posición
    y = np.linspace(agente.low[1], agente.high[1], resolution)  # velocidad
    xx, yy = np.meshgrid(x, y)
    
    policy_map = np.zeros_like(xx)

    for i in range(resolution):
        for j in range(resolution):
            state = np.array([xx[i, j], yy[i, j]])
            policy_map[i, j] = agente._seleccionar_accion_rbf(state)

    plt.figure(figsize=(6, 5))
    plt.contourf(xx, yy, policy_map, levels=agente.nA, cmap="coolwarm", alpha=0.8)
    cbar = plt.colorbar(ticks=range(agente.nA))
    cbar.ax.set_yticklabels([f"A{a}" for a in range(agente.nA)])
    plt.xlabel("Posición")
    plt.ylabel("Velocidad")
    plt.title("Política aprendida (acción elegida en cada punto)")
    plt.grid(True)
    plt.show()

# Objeto AgenteSARSA-SemiGradiente con RBF

In [3]:
class RBFBasisFunctions:
    def __init__(self, low, high, num_centers=(5, 5), sigma=0.2):
        self.low = np.array(low)
        self.high = np.array(high)
        self.num_centers = np.array(num_centers)
        self.sigma = sigma
        
        # Crear centros de las funciones RBF en una grilla regular
        pos_centers = np.linspace(low[0], high[0], num_centers[0])
        vel_centers = np.linspace(low[1], high[1], num_centers[1])
        
        self.centers = []
        for p in pos_centers:
            for v in vel_centers:
                self.centers.append([p, v])
        
        self.centers = np.array(self.centers)
        self.num_features = len(self.centers)
    
    def get_features(self, state):
        """Calcula las características RBF para un estado dado."""
        state = np.array(state)
        features = np.zeros(self.num_features)
        
        for i, center in enumerate(self.centers):
            distance = np.linalg.norm(state - center)
            features[i] = np.exp(-(distance ** 2) / (2 * self.sigma ** 2))
        
        return features

class AgenteSARSARBF:
    def __init__(self, env, alpha=0.1, gamma=0.99, epsilon=1.0, decay=True, num_centers=(5, 5), sigma=0.2):
        self.env = env
        self.alpha = alpha
        self.gamma = gamma
        self.epsilon = epsilon
        self.decay = decay

        self.nA = env.action_space.n
        self.low = env.observation_space.low
        self.high = env.observation_space.high
        self.state_dim = env.observation_space.shape[0]

        # Inicializar funciones RBF
        self.rbf = RBFBasisFunctions(self.low, self.high, num_centers, sigma)
        self.d = self.rbf.num_features

        # Una theta por acción
        self.theta = np.zeros((self.nA, self.d))
        self.stats = []
        self.episode_lengths = []

    def _phi(self, state):
        """Devuelve vector de características RBF para un estado"""
        return self.rbf.get_features(state)

    def _Q(self, state, action):
        """Valor aproximado Q(s, a)"""
        return np.dot(self.theta[action], self._phi(state))

    # -------------------- POLÍTICAS Y SELECCIÓN DE ACCIONES --------------------

    def _seleccionar_accion_rbf(self, state):
        """Selecciona una acción usando la política epsilon-soft explícita."""
        policy = self._epsilon_soft_policy_rbf(state)
        return np.random.choice(np.arange(self.nA), p=policy)

    def _epsilon_soft_policy_rbf(self, state):
        """Devuelve una política epsilon-soft como vector de probabilidades para entornos con RBF."""
        q_values = np.array([self._Q(state, a) for a in range(self.nA)])
        
        # Manejar valores problemáticos
        if np.any(np.isnan(q_values)) or np.any(np.isinf(q_values)):
            q_values = np.nan_to_num(q_values)
            
        policy = np.ones(self.nA) * self.epsilon / self.nA
        best_action = np.argmax(q_values)
        policy[best_action] += 1.0 - self.epsilon
        
        # Asegurar que las probabilidades son válidas
        policy = np.clip(policy, 0, 1)
        if policy.sum() == 0:
            policy = np.ones(self.nA) / self.nA
        else:
            policy /= policy.sum()
            
        return policy

    def entrenar(self, num_episodes=5000, mostrar_barra=True):
        random.seed(SEED)
        np.random.seed(SEED)
    
        acumulador_recompensas = 0.0
    
        for t in tqdm(range(num_episodes), disable=not mostrar_barra):
            if self.decay:
                self.epsilon = min(1.0, 1000.0 / (t + 1))
    
            state, _ = self.env.reset()
            action = self._seleccionar_accion_rbf(state)
            done = False
            total_reward = 0
            pasos = 0
    
            while not done:
                next_state, reward, terminated, truncated, _ = self.env.step(action)
                done = terminated or truncated
                next_action = self._seleccionar_accion_rbf(next_state)
    
                phi = self._phi(state)
    
                # Verificar características válidas
                if np.any(np.isnan(phi)) or np.any(np.isinf(phi)):
                    phi = np.nan_to_num(phi)
    
                q_current = self._Q(state, action)
                q_next = self._Q(next_state, next_action) if not done else 0.0
                target = reward + self.gamma * q_next
                delta = target - q_current
    
                if np.isnan(delta) or np.isinf(delta):
                    delta = 0.0
    
                # Actualización con control de valores extremos
                self.theta[action] += self.alpha * delta * phi
                self.theta[action] = np.clip(self.theta[action], -1e3, 1e3)
    
                state = next_state
                action = next_action
                total_reward += reward
                pasos += 1
    
            self.episode_lengths.append(pasos)
            acumulador_recompensas += total_reward
            self.stats.append(acumulador_recompensas / (t + 1))
    
        return self.theta

In [7]:
def evaluar_configuracion(params, env_name="MountainCar-v0"):
    try:
        alpha, gamma, epsilon, decay, num_centers, sigma = params
        env = gym.make(env_name)
        env.reset(seed=SEED)
        agente = AgenteSARSARBF(
            env, alpha=alpha, gamma=gamma, epsilon=epsilon,
            decay=decay, num_centers=num_centers, sigma=sigma
        )
        agente.entrenar(num_episodes=3000, mostrar_barra=False)
        recompensa_final = np.mean(agente.stats[-100:]) if len(agente.stats) >= 100 else np.mean(agente.stats)
        env.close()
        return (alpha, gamma, epsilon, decay, num_centers, sigma, recompensa_final)
    except Exception as e:
        print(f"Error en configuración {params}: {e}")
        return (alpha, gamma, epsilon, decay, num_centers, sigma, -float('inf'))

def random_search_secuencial(env_name="MountainCar-v0", n_trials=50):
    """Versión secuencial del random search para evitar problemas de multiprocesamiento."""
    # Espacio de búsqueda
    alphas = [0.01, 0.05, 0.1, 0.5]
    gammas = [0.9, 0.95, 1.0]
    epsilons = [0.1, 0.2, 0.3, 0.5]
    num_centers_list = [(3, 3), (5, 5), (7, 7), (10, 10)]
    sigmas = [0.1, 0.2, 0.5, 1.0]

    combinaciones = []

    random.seed(SEED)
    np.random.seed(SEED)

    for _ in range(n_trials):
        a = random.choice(alphas)
        g = random.choice(gammas)
        nc = random.choice(num_centers_list)
        s = random.choice(sigmas)
        decay = random.choice([True, False])
        if decay:
            e = 0.0  # ignorado
        else:
            e = random.choice(epsilons)
        combinaciones.append((a, g, e, decay, nc, s))

    mejor_config = None
    mejor_recompensa = -float('inf')
    resultados = []

    print(f"🔍 Random Search RBF (secuencial): ejecutando {n_trials} configuraciones...\n")

    for i, combo in enumerate(tqdm(combinaciones, desc="Progreso")):
        print(f"Configuración {i+1}/{n_trials}: α={combo[0]}, γ={combo[1]}, decay={combo[3]}, centers={combo[4]}, σ={combo[5]}")
        
        resultado = evaluar_configuracion(combo, env_name)
        alpha, gamma, epsilon, decay_flag, num_centers, sigma, recompensa = resultado
        resultados.append(resultado)

        if recompensa > mejor_recompensa:
            mejor_recompensa = recompensa
            mejor_config = (alpha, gamma, epsilon, decay_flag, num_centers, sigma)
            print(f"  ✅ Nueva mejor configuración! Recompensa: {mejor_recompensa:.4f}")

    print("\n" + "="*60)
    print("✅ MEJOR CONFIGURACIÓN ENCONTRADA:")
    print(f" alpha = {mejor_config[0]}")
    print(f" γ = {mejor_config[1]}")
    print(f" ε = {mejor_config[2]}")
    print(f" decay = {mejor_config[3]}")
    print(f" num_centers = {mejor_config[4]}")
    print(f" sigma = {mejor_config[5]}")
    print(f"  → Recompensa media final: {mejor_recompensa:.4f}")
    print("="*60)
    
    return mejor_config, mejor_recompensa, resultados

def random_search(env_name="MountainCar-v0", n_trials=30):
    """Wrapper que llama a la versión secuencial."""
    return random_search_secuencial(env_name, n_trials)

In [ ]:
# Ejecutar búsqueda de hiperparámetros
print("🚀 Iniciando búsqueda de hiperparámetros para SARSA RBF...")
mejor_config, mejor_recompensa, resultados = random_search(n_trials=30)

print(f"\n📊 Mejores hiperparámetros encontrados:")
print(f"   α = {mejor_config[0]}")
print(f"   γ = {mejor_config[1]}")  
print(f"   ε = {mejor_config[2]}")
print(f"   decay = {mejor_config[3]}")
print(f"   num_centers = {mejor_config[4]}")
print(f"   σ = {mejor_config[5]}")
print(f"   Recompensa final = {mejor_recompensa:.4f}")

🚀 Iniciando búsqueda de hiperparámetros para SARSA RBF...
🔍 Random Search RBF (secuencial): ejecutando 30 configuraciones...



Progreso:   0%|          | 0/30 [00:00<?, ?it/s]

Configuración 1/30: α=0.01, γ=0.9, decay=True, centers=(7, 7), σ=0.2


Progreso:   3%|▎         | 1/30 [54:10<26:11:02, 3250.44s/it]

  ✅ Nueva mejor configuración! Recompensa: -200.0000
Configuración 2/30: α=0.05, γ=1.0, decay=False, centers=(3, 3), σ=0.1


Progreso:   7%|▋         | 2/30 [1:06:41<13:50:38, 1779.95s/it]

  ✅ Nueva mejor configuración! Recompensa: -178.2809
Configuración 3/30: α=0.01, γ=0.9, decay=True, centers=(5, 5), σ=0.2


Progreso:  10%|█         | 3/30 [1:34:17<12:55:33, 1723.45s/it]

Configuración 4/30: α=0.05, γ=1.0, decay=False, centers=(10, 10), σ=0.2


In [ ]:
# Ejecución de ejemplo
env = gym.make("MountainCar-v0")
env.reset(seed=SEED)
agente = AgenteSARSARBF(env, alpha=0.1, gamma=1.0, epsilon=0.3, num_centers=(7, 7), sigma=0.2, decay=False)
agente.entrenar(num_episodes=15000)

# Resultados de agente SARSA SemiGradiente con RBF

In [ ]:
mostrar_resultados_agente_continuo(agente)

In [ ]:
ejecutar_episodio_y_mostrar(agente)

In [ ]:
def grabar_video_agente(agente, nombre_archivo="video_mountaincar.gif", fps=30):
    """
    Ejecuta un episodio con la política aprendida y guarda un video del entorno.
    """
    # Crear entorno con renderizado de imágenes
    env = gym.make("MountainCar-v0", render_mode="rgb_array")
    env.reset(seed=SEED)
    state, _ = env.reset()
    done = False
    total_reward = 0
    frames = []

    while not done:
        frame = env.render()
        frames.append(frame)

        # Acción greedy
        q_values = [agente._Q(state, a) for a in range(agente.nA)]
        action = np.argmax(q_values)

        next_state, reward, terminated, truncated, _ = env.step(action)
        total_reward += reward
        done = terminated or truncated
        state = next_state

        if done:
            break

    env.close()

    print(f"Número total de frames: {len(frames)}")
    # Guardar el video como GIF
    imageio.mimsave(nombre_archivo, frames, fps=fps, loop=0)
    print(f"🎥 Vídeo guardado en: {nombre_archivo}")
    print(f"🏁 Recompensa total obtenida: {total_reward:.2f}")

In [ ]:
nombre_archivo = "video_mountaincar_rbf.gif"

grabar_video_agente(agente, nombre_archivo=nombre_archivo)

HTML(f"""
<img src="{nombre_archivo}" style="width: 600px;" loop>
""")